# Generating results

This notebook is associated with:
* the manuscript "Non-parametric power-law surrogates" which currently (2021-12-14) is under review at PRX;
* the module constrained_power_law_surogates; and
* the repository https://github.com/JackMurdochMoore/power-law .

This notebook is designed to illustrate generation of most results plotted in the figures and tables of the manuscript, but does not produce the figures themselves.

By default, we:
* look for time series in the folder *time-series* (in the directory which contains the current directory); and
* save results in the folder *results* (in the directory which contains the current directory)

The code is provided without warranty or guarantee.

In [1]:
# Import module constrained_power_law_surrogates
#
import sys
sys.path.append("../src/")
from constrained_power_law_surrogates import *

results_folder_name = 'results-test'#Use non-default name for results folder when we do not want to overwrite existing results

# If folders ts_folder_name and results_folder_name do not exist then make them
import os
# cwd = os.getcwd()
os.makedirs('../' + ts_folder_name + '/', exist_ok=True)
os.makedirs('../' + results_folder_name + '/', exist_ok=True)

In [2]:
# Generate data for Tab. II
# 
# Calculate p-values for iid power-law hypothesis for observed data.
# 
num_surr = 9;

# Comment/uncomment the following to load different text data on which to perform hypothesis tests：
# 
# file_name_str = 'flares.txt'; #x_min = 323, N = 12773, NG = 1711
# file_name_str = 'blackouts.txt'; #x_min = 230;#+/-90, N = 211, NG = 57
# file_name_str = 'terrorism.txt'; #x_min = 12;#+/-4, N = 9,101, NG = 547
# file_name_str = 'words.txt'; #x_min = 7;#+/-2, N = 18,855, NG = 958
# file_name_str = 'rescaled-diseases.txt'; #x_min = 2317, N = 72, NG = 27
file_name_str = 'normed-flares.txt'; #x_min = 1, N = 1711, NG = 1711
# file_name_str = 'energy.txt'; #x_min = 1, N = 59555, NG = 59555#Approximate energies of earthquakes in Southern California

data = np.loadtxt('../' + ts_folder_name + '/' + file_name_str)

if (file_name_str == 'blackouts.txt'):
    data = [round(val/1000) for val in data]#Some elements of this sequence of data seem to be less precise than others; use units of 1000 and round to nearest integers so that each has the same precision

N_full = len(data)

print('{} data.'.format(N_full))
#
# Make some parameter choices:

stat_code_list = [0, 1, 2, 3, 4]#[KS-distance, variance, mean, maximum, conditional entropy of order one]

num_stat_codes = len(stat_code_list)

for zero_for_con_one_for_typ in [0, 1]:#0 - use constrained surrogates, 1 - use typical surrogates
    start_time = timer()

    if (zero_for_con_one_for_typ == 0):
        con_typ_str = 'const'
    elif (zero_for_con_one_for_typ == 1):
        con_typ_str = 'typic'

    print('Full N = {}, stat codes = {}, surr. = {}.'.format(N_full, stat_code_list, con_typ_str))

    (x_min, x_min_list, ks_dist_list, gamma_m_l, gamma_m_l_list) = ident_cut_off_const(data)
    obs_seq = [int(val) for val in data if val >= x_min]
    obs_seq_0 = [int(val) for val in data if val < x_min]

    N = len(obs_seq)
    N_0 = len(obs_seq_0)

    print('{} {} surrogates, min. deg = {}, N = {}.'.format(num_surr, con_typ_str, x_min, N))

    #Find rejection rates:
    p_val_list = np.zeros((num_stat_codes))
    start_current_num_nodes_time = timer()
    obs_stat_list = np.zeros(num_stat_codes)
    for ii_stat in range(num_stat_codes):
        obs_stat_list[ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](obs_seq, x_min, gamma_m_l)
    surr_stat_list_list = np.zeros((num_surr, num_stat_codes))
    surr_stat_list = np.zeros(num_surr)
    x_min_surr_list = []
    if (zero_for_con_one_for_typ == 0):
        gamma_m_l_surr = gamma_m_l
        obs_factor_dict = build_factor_count_dict(obs_seq)
        if (x_min == 1):
            for ii in range(num_surr):
                valid_surr_flag = False
                while (not valid_surr_flag):
                    surr_val_seq = gen_cons_power_law_surr_1(obs_seq, obs_factor_dict)
                    surr_factor_dict = build_factor_count_dict(surr_val_seq)
                    (x_min_surr, x_min_list, ks_dist_list, gamma_m_l, gamma_m_l_list) = ident_cut_off_const(surr_val_seq)
                    if not operator.eq(obs_factor_dict, surr_factor_dict):
                        set1 = set(obs_factor_dict.items())
                        set2 = set(surr_factor_dict.items())
    #                     print('There is a problem: prime factor counts of surrogate do not match those of original. Difference:')
    #                     print(set1 ^ set2)
    #                     print('We will generate a replacement surrogate.')
                        valid_surr_flag = False
                    elif not (x_min == x_min_surr):
    #                     print('Orig. min. deg. orig. = {}, min. deg. surr. #{} = {}'.format(x_min, ii, x_min_surr))
    #                     print('We will generate a replacement surrogate.')
                        valid_surr_flag = False
                        x_min_surr_list = x_min_surr_list + [x_min_surr]
                        x_min_surr_list = list(set(x_min_surr_list))
                        x_min_surr_list.sort()
                    else:
                        valid_surr_flag = True
                        print('Surr. #{} is valid.'.format(ii))
                    for ii_stat in range(num_stat_codes):
                        surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
        else:
            (move_factor_dict, factor_list, min_admiss_factor_ind_list, val_fix_list) = make_move_factor_dict(obs_factor_dict, obs_seq, x_min)
            for ii in range(num_surr):
                valid_surr_flag = False
                while (not valid_surr_flag):
                    surr_val_seq = gen_cons_power_law_surr_2(move_factor_dict, factor_list, min_admiss_factor_ind_list, val_fix_list)
                    (x_min_surr, x_min_list, ks_dist_list, gamma_m_l, gamma_m_l_list) = ident_cut_off_const(obs_seq_0 + surr_val_seq)
                    surr_factor_dict = build_factor_count_dict(surr_val_seq)
                    if not operator.eq(obs_factor_dict, surr_factor_dict):
                        set1 = set(obs_factor_dict.items())
                        set2 = set(surr_factor_dict.items())
    #                     print('There is a problem: prime factor counts of surrogate do not match those of original. Difference:')
    #                     print(set1 ^ set2)
    #                     print('We will generate a replacement surrogate.')
                    elif not (x_min == x_min_surr):
    #                     print('Orig. min. deg. orig. = {}, min. deg. surr. #{} = {}'.format(x_min, ii, x_min_surr))
    #                     print('We will generate a replacement surrogate.')
                        valid_surr_flag = False
                        x_min_surr_list = x_min_surr_list + [x_min_surr]
                        x_min_surr_list = list(set(x_min_surr_list))
                        x_min_surr_list.sort()
                    else:
                        valid_surr_flag = True
                        print('Surr. #{} is valid.'.format(ii))
                for ii_stat in range(num_stat_codes):
                    surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                surr_factor_dict = build_factor_count_dict(surr_val_seq)
    elif (zero_for_con_one_for_typ == 1):
        p_high = N/(N + N_0)
        for ii in range(num_surr):
            N_surr = int(np.random.binomial(N + N_0, p_high, 1))
            N_0_surr = N + N_0 - N_surr
            surr_val_seq = list(np.random.choice(obs_seq_0, N_0_surr)) + gen_typ_p_l_surrogate(gamma_m_l, x_min, N_surr)
            (x_min_surr, x_min_list, ks_dist_list, gamma_m_l_surr, gamma_m_l_surr_list) = ident_cut_off_const(surr_val_seq)
            surr_val_seq = [int(val) for val in surr_val_seq if val >= x_min_surr]
            for ii_stat in range(num_stat_codes):
                surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min_surr, gamma_m_l_surr)
    for ii_stat in range(num_stat_codes):
        surr_stat_list = surr_stat_list_list[:, ii_stat]
        stat_factor = stat_factor_dict[stat_code_list[ii_stat]]
        surr_stat_list = np.multiply(stat_factor, surr_stat_list)
        obs_stat = np.multiply(stat_factor, obs_stat_list[ii_stat])
        rankL = sum(surr_stat_list > obs_stat)
        rankU = sum(surr_stat_list >= obs_stat)
        rank = 1 + random.randint(rankL, rankU)
        p_val_list[ii_stat] = rank/(num_surr + 1)
    end_current_num_nodes_time = timer()
    print('Rejection rates (N = {}):'.format(N))
    print(np.transpose(p_val_list))
    print('Min. deg. = {}, N = {} took {} sec.'.format(x_min, N, end_current_num_nodes_time - start_current_num_nodes_time))

    end_time = timer()
    total_time = end_time - start_time
    print('N = {} took {} sec.'.format(N, total_time))

    file_name_str = 'full_p_l_test_{}_N-{}_KS_var_mean_max_surr-{}_{}surr_safe-{}'.format(file_name_str[:-4], N, con_typ_str, num_surr, 1).replace(".", "-")
    np.savetxt('../' + results_folder_name + '/' + file_name_str + '_dir_p_vals_test.dat', p_val_list)

1711 data.
Full N = 1711, stat codes = [0, 1, 2, 3, 4], surr. = const.
9 const surrogates, min. deg = 1, N = 1711.
Surr. #0 is valid.
Surr. #1 is valid.
Surr. #2 is valid.
Surr. #3 is valid.
Surr. #4 is valid.
Surr. #5 is valid.
Surr. #6 is valid.
Surr. #7 is valid.
Surr. #8 is valid.
Rejection rates (N = 1711):
[0.1 0.1 0.1 0.1 0.1]
Min. deg. = 1, N = 1711 took 10.602745299999999 sec.
N = 1711 took 12.091121399999999 sec.
Full N = 1711, stat codes = [0, 1, 2, 3, 4], surr. = typic.
9 typic surrogates, min. deg = 1, N = 1711.
Rejection rates (N = 1711):
[0.1 0.1 0.1 0.1 0.1]
Min. deg. = 1, N = 1711 took 9.879690799999999 sec.
N = 1711 took 11.096005400000003 sec.


In [3]:
# Generate data for Fig. 1
# 
# Prepare data for a plot comparing power-law, log-normal and correlated power-law
# 
max_val = np.inf
gamma_m_l_corr_2 = 2.5
while (max_val >= 130):# Encourage results to fit conveniently in Fig. 1
    N = 1024; min_val = 12;
    gamma = 2.5;
    mu = 0; sigma = 1.5; 

    name_str = 'power-law-log-norm-max-like';#0: Power-law, 2: Log-normal, mu = 0

    #IID power-law and IID log-normal sequences
    val_seq_power_law = gen_typ_p_l_surrogate(gamma, min_val, N)
    val_seq_log_normal = gen_log_norm(sigma, min_val, N)

    plot_data_uncorr = [list(val_seq_power_law)] + [list(val_seq_log_normal)]
    np.save('../' + results_folder_name + '/' + name_str + '_uncorr', plot_data_uncorr)

    b = 3
    mu = 0.1#Probability of randomly choosing from limiting distribution

    val_seq_power_law_corr_2 = gen_mc_pl_seq(N, gamma, min_val, b, mu)

    np.save('../' + results_folder_name + '/' + name_str + '_correlated_2', [val_seq_power_law_corr_2])

    x_min = min([min(val_seq_power_law), min(val_seq_log_normal), min(val_seq_power_law_corr_2)])
    x_max = max([max(val_seq_power_law), max(val_seq_log_normal), max(val_seq_power_law_corr_2)])

    #Observed pdfs for correlated power-law and IID log-normal sequences (panel a)
    bins = np.arange(x_min - 0.5, x_max + 0.5 + 10**-10, 1)
    bin_mids = 0.5*(bins[:-1] + bins[1:])
    bin_widths = (bins[1:] - bins[:-1])

    #Observed pdfs for IID power-law and IID log-normal sequences (panel a)
    bins = np.arange(x_min - 0.5, x_max + 0.5 + 10**-10, 1)
    bin_mids = 0.5*(bins[:-1] + bins[1:])
    bin_widths = (bins[1:] - bins[:-1])
    emp_pdf_power_law = calc_emp_pdf(val_seq_power_law, bins)
    emp_pdf_log_normal = calc_emp_pdf(val_seq_log_normal, bins)

    xx = [x for x in range(min_val, int(1.2*np.floor((bins[-1:]))))]

    #Power-law and log-normal pmfs (panels a and b)
    inv_norm = zeta(gamma, min_val);
    yy_power_law = [x**-gamma/inv_norm for x in xx];
    def unnormed_log_norm_cdf(mu, sigma, x):
        return (0.5 + 0.5*erf((np.log(x) - mu)/(np.sqrt(2)*sigma)))
    inv_norm = 1 - unnormed_log_norm_cdf(mu, sigma, min_val - 0.5)
    yy_log_norm = [(unnormed_log_norm_cdf(mu, sigma, x + 0.5) - unnormed_log_norm_cdf(mu, sigma, x - 0.5))/inv_norm for x in xx]
    yy_list = [yy_power_law, yy_log_norm]

    #Maximum likelihood power-law pmfs (panel b)
    gamma_m_l = estim_scale_exp(val_seq_power_law, min_val)
    inv_norm = zeta(gamma_m_l, min_val);
    yy_power_law_m_l = [x**-gamma_m_l/inv_norm for x in xx]

    plot_data_empir = [list(bin_mids)] + [list(emp_pdf_power_law)] + [list(emp_pdf_log_normal)]
    plot_data_param = [list(xx)] + [list(yy_power_law)] + [list(yy_log_norm)] + [list(yy_power_law_m_l)]
    np.save('../' + results_folder_name + '/' + name_str + '_empir', plot_data_empir)
    np.save('../' + results_folder_name + '/' + name_str + '_param', plot_data_param)    

    emp_pdf_power_law_corr_2 = calc_emp_pdf(val_seq_power_law_corr_2, bins)

    xx = [x for x in range(min_val, int(1.2*np.floor((bins[-1:]))))]

    #Maximum likelihood power-law pmfs (panel b)
    gamma_m_l_corr_2 = estim_scale_exp(val_seq_power_law_corr_2, min_val)
    inv_norm_corr_2 = zeta(gamma_m_l_corr_2, min_val);
    yy_power_law_m_l_corr_2 = [x**-gamma_m_l_corr_2/inv_norm_corr_2 for x in xx]

    plot_data_empir_corr_2 = [list(emp_pdf_power_law_corr_2)]
    plot_data_param_corr_2 =[list(yy_power_law_m_l_corr_2)]
    np.save('../' + results_folder_name + '/' + name_str + '_empir_corr_2', plot_data_empir_corr_2)
    np.save('../' + results_folder_name + '/' + name_str + '_param_corr_2', plot_data_param_corr_2)
    max_val = np.max(val_seq_power_law_corr_2)

In [4]:
# Figures 2-3 are schematics

In [5]:
# Generate data for Fig. 4
#
# Calculate p-values
#
for surr_type_code in [-2, 0, 1]:#surr_type_code = -2 for shuffle, 0 for constrained, 1 for typical
    num_tests_list = [10**1]
    nom_size_list = [0.1]; num_surr = 9;
    x_min_list = [1, 12]
    stat_code_list = [0, 1, 2, 3, 4]#[KS-distance, variance, mean, maximum, conditional entropy]
    
    dist_code = 10; zero_for_size_one_for_power = 1; gamma_list = np.arange(1.5, 1.5 + 10**-6, 0.25);
    
    net_size_list = [2**(n + 1) for n in range(1, 4, 1)];

    num_stat_codes = len(stat_code_list)

    for num_tests in num_tests_list:
        for nom_size in nom_size_list:
            for x_min in x_min_list:
                # Make more parameter choices:
                start_time = timer()

                if (zero_for_size_one_for_power == 0):
                    dist_code = 0;
                    gamma_list = [gamma for gamma in gamma_list if gamma > 1]# The power-law distribution is not defined for scale exponent less than or equal to 1
                    siz_pow_str = 'siz'
                elif (zero_for_size_one_for_power == 1):
                    siz_pow_str = 'pow'

                if (surr_type_code == -1):
                    gamma_list = [gamma for gamma in gamma_list if gamma > 1]# The power-law distribution is not defined for scale exponent less than or equal to 1

                if (surr_type_code == -2):
                    con_typ_str = 'shuff'
                elif (surr_type_code == 0):
                    con_typ_str = 'const'
                elif (surr_type_code == 1):
                    con_typ_str = 'typic'

                gamma_list = list(gamma_list); num_diff_gamma = len(gamma_list)
                num_diff_size = len(net_size_list);

                print('{} tests, {} {} surrogates, min. deg = {}.'.format(num_tests, num_surr, con_typ_str, x_min))    
                print('gamma = {}, N = {}, stat codes = {}, surr. = {}, rej. = {}, distrib. = {}.'.format(gamma_list, net_size_list, stat_code_list, con_typ_str, siz_pow_str, dist_code))

                #Find rejection rates:
                rej_rate_list_list_list = np.zeros((num_diff_gamma, num_diff_size, num_stat_codes))
                all_p_vals = np.zeros((num_stat_codes, num_diff_gamma, num_diff_size, num_tests))
                for ll in range(num_diff_gamma):
                    gamma = gamma_list[ll]
                    start_time_current_gamma = timer()
                    for kk in range(num_diff_size):
                        N = net_size_list[kk]
                        start_current_num_nodes_time = timer()
                        num_rej_list = np.zeros(num_stat_codes)
                        for jj in range(num_tests):
                            (obs_seq, dist_str) = gen_orig(dist_code, gamma, x_min, N, zero_for_size_one_for_power)
                            gamma_m_l = estim_scale_exp(obs_seq, x_min)
                            obs_stat_list = np.zeros(num_stat_codes)
                            for ii_stat in range(num_stat_codes):
                                obs_stat_list[ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](obs_seq, x_min, gamma_m_l)
                            surr_stat_list_list = np.zeros((num_surr, num_stat_codes))
                            surr_stat_list = np.zeros(num_surr)
                            if (surr_type_code == -2):
                                for ii in range(num_surr):
                                    surr_val_seq = list(np.random.permutation(obs_seq))
                                    gamma_m_l_surr = gamma_m_l
                                    for ii_stat in range(num_stat_codes):
                                        surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                            elif (surr_type_code == -1):
                                for ii in range(num_surr):
                                    surr_val_seq = gen_typ_p_l_surrogate(gamma, x_min, N)
                                    gamma_m_l_surr = estim_scale_exp(surr_val_seq, x_min)
                                    for ii_stat in range(num_stat_codes):
                                        surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                            elif (surr_type_code == 0):
                                gamma_m_l_surr = gamma_m_l
                                obs_factor_dict = build_factor_count_dict(obs_seq)
                                if (x_min == 1):
                                    for ii in range(num_surr):
                                        valid_surr_flag = False
                                        while (not valid_surr_flag):
                                            surr_val_seq = gen_cons_power_law_surr_1(obs_seq, obs_factor_dict)
                                            for ii_stat in range(num_stat_codes):
                                                surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                            surr_factor_dict = build_factor_count_dict(surr_val_seq)
                                            if not operator.eq(obs_factor_dict, surr_factor_dict):
                                                set1 = set(obs_factor_dict.items())
                                                set2 = set(surr_factor_dict.items())
                                                print('There is a problem: prime factor counts of surrogate do not match those of original. Difference:')
                                                print(set1 ^ set2)
                                                print('We will generate a replacement surrogate.')
                                                valid_surr_flag = False
                                            else:
                                                valid_surr_flag = True
                                else:
                                    (move_factor_dict, factor_list, min_admiss_factor_ind_list, val_fix_list) = make_move_factor_dict(obs_factor_dict, obs_seq, x_min)
                                    for ii in range(num_surr):
                                        valid_surr_flag = False
                                        while (not valid_surr_flag):
                                            surr_val_seq = gen_cons_power_law_surr_2(move_factor_dict, factor_list, min_admiss_factor_ind_list, val_fix_list)
                                            for ii_stat in range(num_stat_codes):
                                                surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                            surr_factor_dict = build_factor_count_dict(surr_val_seq)
                                            if not operator.eq(obs_factor_dict, surr_factor_dict):
                                                set1 = set(obs_factor_dict.items())
                                                set2 = set(surr_factor_dict.items())
                                                print('There is a problem: prime factor counts of surrogate do not match those of original. Difference:')
                                                print(set1 ^ set2)
                                                print('We will generate a replacement surrogate.')
                                                valid_surr_flag = False
                                            else:
                                                valid_surr_flag = True
                            elif (surr_type_code == 1):
                                for ii in range(num_surr):
                                    surr_val_seq = gen_typ_p_l_surrogate(gamma_m_l, x_min, N)
                                    gamma_m_l_surr = estim_scale_exp(surr_val_seq, x_min)
                                    for ii_stat in range(num_stat_codes):
                                        surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                            for ii_stat in range(num_stat_codes):
                                surr_stat_list = surr_stat_list_list[:, ii_stat]
                                stat_factor = stat_factor_dict[stat_code_list[ii_stat]]
                                surr_stat_list = np.multiply(stat_factor, surr_stat_list)
                                obs_stat = np.multiply(stat_factor, obs_stat_list[ii_stat])
                                rankL = sum(surr_stat_list > obs_stat)
                                rankU = sum(surr_stat_list >= obs_stat)
                                rank = 1 + random.randint(rankL, rankU)
                                p_val = (rank - 0.5)/(num_surr + 1)
                                if (p_val <= nom_size):
                                    num_rej_list[ii_stat] += 1
                                all_p_vals[ii_stat, ll, kk, jj] = p_val
                        for ii_stat in range(num_stat_codes):
                            rej_rate_list_list_list[ll, kk, ii_stat] = num_rej_list[ii_stat]/num_tests
                        end_current_num_nodes_time = timer()
                        if (ll == 0):
                            print('N = {} took {} sec.'.format(N, end_current_num_nodes_time - start_current_num_nodes_time))
                    end_time_current_gamma = timer()
                    total_time_current_gamma = end_time_current_gamma - start_time_current_gamma
                    print('gamma = {}, N = {} took {} sec, rej. rates:'.format(gamma, net_size_list, total_time_current_gamma))
                    print(np.transpose(rej_rate_list_list_list[ll, :, :]))
                end_time = timer()
                total_time = end_time - start_time
                print('N = {} took {} sec.'.format(net_size_list, total_time))

                for ii_stat in range(num_stat_codes):
                    stat_code = stat_code_list[ii_stat]
                    stat_str = stat_str_dict[stat_code]
                    rej_rate_list_list = rej_rate_list_list_list[:, :, ii_stat]
                    #Save result in a text file:
                    file_name_str = '{}gamma-{}-{}-{}_x_min-{}_{}N-{}-{}_stat-{}_surr-{}_rej-{}_{}_nom_size-{}_{}surr_{}tests_safe-{}'.format(len(gamma_list), min(gamma_list), max(gamma_list), gamma, x_min, len(net_size_list), min(net_size_list), max(net_size_list), stat_str, con_typ_str, siz_pow_str, dist_str, nom_size, num_surr, num_tests, 1).replace(".", "-")
                    rel_param_list = [x_min, num_diff_size, num_diff_gamma, stat_code, zero_for_size_one_for_power, dist_code, surr_type_code, nom_size, num_surr, num_tests, start_time, total_time]
                    num_rel_param = len(rel_param_list)
                    length_longest_vec = max(num_diff_gamma, num_diff_size, num_rel_param)
                    data = np.column_stack((list(np.int_(net_size_list)) + [0]*(length_longest_vec - num_diff_size), list(gamma_list) + [0]*(length_longest_vec - num_diff_gamma),  rel_param_list + [0]*(length_longest_vec - num_rel_param)))
                    header = 'Num. nodes, gamma, x_min-num_diff_size-num_diff_gamma-stat_code-zero_for_size_one_for_power-dist_code-surr_type_code-nom_size-num_surr-num_tests-start_time-total_time'
                    np.savetxt('../' + results_folder_name + '/' + file_name_str + '_dir_param_test.dat', data, header=header)
                    np.save('../' + results_folder_name + '/' + file_name_str + '_dir_p_vals_test', all_p_vals[ii_stat])
                    np.savetxt('../' + results_folder_name + '/' + file_name_str + '_dir_rej_rates_test.dat', rej_rate_list_list)

10 tests, 9 shuff surrogates, min. deg = 1.
gamma = [1.5], N = [4, 8, 16], stat codes = [0, 1, 2, 3, 4], surr. = shuff, rej. = pow, distrib. = 10.
N = 4 took 0.18435799999999603 sec.
N = 8 took 0.18565929999999753 sec.
N = 16 took 0.1811888999999951 sec.
gamma = 1.5, N = [4, 8, 16] took 0.5513635000000008 sec, rej. rates:
[[0.1 0.2 0. ]
 [0.  0.1 0.1]
 [0.  0.  0. ]
 [0.  0.1 0. ]
 [0.1 0.  0. ]]
N = [4, 8, 16] took 0.5516397999999967 sec.
10 tests, 9 shuff surrogates, min. deg = 12.
gamma = [1.5], N = [4, 8, 16], stat codes = [0, 1, 2, 3, 4], surr. = shuff, rej. = pow, distrib. = 10.
N = 4 took 0.18021639999999906 sec.
N = 8 took 0.18252640000000042 sec.
N = 16 took 0.18608520000000084 sec.
gamma = 1.5, N = [4, 8, 16] took 0.5489938000000052 sec, rej. rates:
[[0.1 0.1 0. ]
 [0.2 0.5 0.2]
 [0.1 0.1 0. ]
 [0.2 0.1 0.1]
 [0.  0.1 0. ]]
N = [4, 8, 16] took 0.5492571999999996 sec.
10 tests, 9 const surrogates, min. deg = 1.
gamma = [1.5], N = [4, 8, 16], stat codes = [0, 1, 2, 3, 4], surr.

In [6]:
# Generate data for Fig. 5-6
# 
# Calculate rate of rejection of power-law hypothesis with tests based on a chosen surrogate type.
#
# Make some parameter choices:
L = 16# Length of ordinal patterns
o = 1# Markov order
b = 3# Bin size
num_trans = 10**5# Number of transitions when using constrained Markov or ordinal pattern power-law surrogates
for dist_code in [0, 10]:
    for surr_type_code in [17, 20]:#-2 for shuffle, -1 for unrealistic (power-law with true cut-off and gamma), 0 for constrained, 1 for typical, 2 for constrained power-law with bins of Markov order one, 3 for typical power-law with bins of Markov order one, 4 for constrained power-law with bins of Markov order zero, 5 for typical power-law with bins of Markov order zero
        num_tests_list = [10**1]
        nom_size_list = [0.1]; num_surr = 9;
        x_min_list = [1]
        stat_code_list = [0, 1, 2, 3, 4, 20, 21]#KS distance, variance, mean, maximum, conditional entropy, coefficient of variation, order 2 conditional entropy

        # NMax = np.Inf
        NMax = 64
        
        zero_for_size_one_for_power = 1
        
        if dist_code == 0:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);zero_for_size_one_for_power = 0;#Discrete power-law
        elif dist_code == 10:
            gamma_list = np.arange(1.5, 1.5 + 10**-6, 0.5);#Discretised conditioned log-normal distribution with mu = 0
        elif dist_code == 17:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#Power law with exponential cutoff defined by lam = Lambda = lambda = 0.01
        elif dist_code == 25:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); x_min_list = [1];#Discrete power-law which is a Markov chain of order one with a 90% chance that proposed value is in the same bin as the current value
        elif dist_code == 26:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); x_min_list = [1];#Discrete power-law which is a Markov chain of order two with a 90% chance that proposed value is in the same bin as one of the two most recent values
        elif dist_code == 35:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); x_min_list = [1];#Extract of sequential earthquake energies in Southern California
        elif dist_code == 57:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); NMax = 27; x_min_list = [2317]; #N = 27, NG = 27, Extract of tail of rescaled numbers of fatalities in epidemics
        elif dist_code == 58:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); NMax = 57; x_min_list = [235]; #N = 57, NG = 57, Extract of tail of thousands of people affected by blackouts
        elif dist_code == 59:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); NMax = 1711; x_min_list = [1]; #N = 1711, NG = 1711, Extract of renormalised tail of peak gamma-ray intensity of solar flares between 1980 and 1989

        if (NMax > 1024):
            NMax = 1024
        numN = int(np.round(np.log2(NMax))) - 1;
        numN = 9;

        indNList = 2 + np.arange(0, numN, 1)*(np.log2(NMax) - 2)/(numN - 1)
        net_size_list = [int(np.round(2**n)) for n in indNList]

        num_stat_codes = len(stat_code_list)

        for num_tests in num_tests_list:
            for nom_size in nom_size_list:
                for x_min in x_min_list:
                    # Make more parameter choices:
                    start_time = timer()

                    if (zero_for_size_one_for_power == 0):
                        dist_code = 0;
                        gamma_list = [gamma for gamma in gamma_list if gamma > 1]# The power-law distribution is not defined for scale exponent less than or equal to 1
                        siz_pow_str = 'siz'
                    elif (zero_for_size_one_for_power == 1):
                        siz_pow_str = 'pow'

                    if (surr_type_code == -1):
                        gamma_list = [gamma for gamma in gamma_list if gamma > 1]# The power-law distribution is not defined for scale exponent less than or equal to 1

                    if (surr_type_code == -2):#Shuffle
                        con_typ_str = 'shuff'
                    elif (surr_type_code == -1):#Generate from power-law with true exponent gamma
                        con_typ_str = 'unrea'
                    elif (surr_type_code == 0):#Constrained power-law surrogate
                        con_typ_str = 'const'
                    elif (surr_type_code == 1):#Typical power-law surrogate
                        con_typ_str = 'typic'
                    elif (surr_type_code == 17):#Constrained power-law and also constrained in terms of ordinal patterns of length L (this version allows consideration of elements in the same ordinal pattern)
                        print('L = ' + str(L));
                        con_typ_str = 'conoplt' + str(L) + 'tr' + str(num_trans)
                    elif (surr_type_code == 20):#Constrained power-law and also constrained in terms of relaxed ordinal patterns of length L (this version allows consideration of elements in the same ordinal pattern)
                        print('L = ' + str(L));
                        con_typ_str = 'mcconb' + str(b) + 'o' + str(o) + 'tr' + str(num_trans)

                    gamma_list = list(gamma_list); num_diff_gamma = len(gamma_list)
                    num_diff_size = len(net_size_list);

                    print('{} tests, {} {} surrogates, min. deg = {}.'.format(num_tests, num_surr, con_typ_str, x_min))    
                    print('gamma = {}, N = {}, stat codes = {}, surr. = {}, rej. = {}, distrib. = {}.'.format(gamma_list, net_size_list, stat_code_list, con_typ_str, siz_pow_str, dist_code))

                    #Find rejection rates:
                    rej_rate_list_list_list = np.zeros((num_diff_gamma, num_diff_size, num_stat_codes))
                    for ll in range(num_diff_gamma):
                        gamma = gamma_list[ll]
                        start_time_current_gamma = timer()
                        for kk in range(num_diff_size):
                            N = net_size_list[kk]
                            start_current_num_nodes_time = timer()
                            num_rej_list = np.zeros(num_stat_codes)
                            for jj in range(num_tests):
                                (obs_seq, dist_str) = gen_orig(dist_code, gamma, x_min, N, zero_for_size_one_for_power)
                                gamma_m_l = estim_scale_exp(obs_seq, x_min)
                                obs_stat_list = np.zeros(num_stat_codes)
                                for ii_stat in range(num_stat_codes):
                                    obs_stat_list[ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](obs_seq, x_min, gamma_m_l)
                                surr_stat_list_list = np.zeros((num_surr, num_stat_codes))
                                surr_stat_list = np.zeros(num_surr)
                                if (surr_type_code == -2):
                                    for ii in range(num_surr):
                                        surr_val_seq = list(np.random.permutation(obs_seq))
                                        gamma_m_l_surr = gamma_m_l
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == -1):
                                    for ii in range(num_surr):
                                        surr_val_seq = gen_typ_p_l_surrogate(gamma, x_min, N)
                                        gamma_m_l_surr = estim_scale_exp(surr_val_seq, x_min)
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == 0):
                                    gamma_m_l_surr = gamma_m_l
                                    obs_factor_dict = build_factor_count_dict(obs_seq)
                                    if (x_min == 1):
                                        for ii in range(num_surr):
                                            valid_surr_flag = False
                                            while (not valid_surr_flag):
                                                surr_val_seq = gen_cons_power_law_surr_1(obs_seq, obs_factor_dict)
                                                for ii_stat in range(num_stat_codes):
                                                    surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                                surr_factor_dict = build_factor_count_dict(surr_val_seq)
                                                if not operator.eq(obs_factor_dict, surr_factor_dict):
                                                    set1 = set(obs_factor_dict.items())
                                                    set2 = set(surr_factor_dict.items())
                                                    print('There is a problem: prime factor counts of surrogate do not match those of original. Difference:')
                                                    print(set1 ^ set2)
                                                    print('We will generate a replacement surrogate.')
                                                    valid_surr_flag = False
                                                else:
                                                    valid_surr_flag = True
                                    else:
                                        (move_factor_dict, factor_list, min_admiss_factor_ind_list, val_fix_list) = make_move_factor_dict(obs_factor_dict, obs_seq, x_min)
                                        for ii in range(num_surr):
                                            valid_surr_flag = False
                                            while (not valid_surr_flag):
                                                surr_val_seq = gen_cons_power_law_surr_2(move_factor_dict, factor_list, min_admiss_factor_ind_list, val_fix_list)
                                                for ii_stat in range(num_stat_codes):
                                                    surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                                surr_factor_dict = build_factor_count_dict(surr_val_seq)
                                                if not operator.eq(obs_factor_dict, surr_factor_dict):
                                                    set1 = set(obs_factor_dict.items())
                                                    set2 = set(surr_factor_dict.items())
                                                    print('There is a problem: prime factor counts of surrogate do not match those of original. Difference:')
                                                    print(set1 ^ set2)
                                                    print('We will generate a replacement surrogate.')
                                                    valid_surr_flag = False
                                                else:
                                                    valid_surr_flag = True
                                elif (surr_type_code == 1):
                                    for ii in range(num_surr):
                                        surr_val_seq = gen_typ_p_l_surrogate(gamma_m_l, x_min, N)
                                        gamma_m_l_surr = estim_scale_exp(surr_val_seq, x_min)
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == 17):
                                    gamma_m_l_surr = gamma_m_l
                                    surr_val_seq = obs_seq
                                    for ii in range(num_surr):
                                        surr_val_seq = gen_p_l_markov_o_p_metr_hast_L(surr_val_seq, num_trans, x_min, L)
                                        gamma_m_l_surr = gamma_m_l
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == 20):
                                    bin_edges, bin_seq, low_lim_list, upp_lim_list = log_bin(obs_seq, b, x_min)
                                    surr_val_seq_list = gen_mc_pl_surrogate(obs_seq, bin_edges, num_trans, o, num_surr)
                                    for ii in range(num_surr):
                                        surr_val_seq = surr_val_seq_list[ii]
                                        gamma_m_l_surr = gamma_m_l
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                for ii_stat in range(num_stat_codes):
                                    surr_stat_list = surr_stat_list_list[:, ii_stat]
                                    stat_factor = stat_factor_dict[stat_code_list[ii_stat]]
                                    obs_stat = np.multiply(stat_factor, obs_stat_list[ii_stat])
                                    abs_obs_stat = abs(obs_stat)
                                    obs_stat = obs_stat + 10**-6*abs_obs_stat*(np.random.uniform(size=1) - 0.5)#Add a small random perturbation to avoid ties
                                    surr_stat_list = np.multiply(stat_factor, surr_stat_list) + 10**-6*abs_obs_stat*(np.random.uniform(size=num_surr) - 0.5)#Add small random perturbations to avoid ties
                                    rankL = sum(surr_stat_list > obs_stat);
                                    rankU = sum(surr_stat_list >= obs_stat)
                                    rank = 1 + random.randint(rankL, rankU)#Randomly assign ranks among tied values
                                    p_val = (rank - 0.5)/(num_surr + 1)
                                    if (p_val <= nom_size):
                                        num_rej_list[ii_stat] += 1
                            for ii_stat in range(num_stat_codes):
                                rej_rate_list_list_list[ll, kk, ii_stat] = num_rej_list[ii_stat]/num_tests
                            end_current_num_nodes_time = timer()
                            if (ll == 0):
                                print('N = {} took {} sec.'.format(N, end_current_num_nodes_time - start_current_num_nodes_time))
                                print(np.transpose(rej_rate_list_list_list[ll, kk, :]))
                        end_time_current_gamma = timer()
                        total_time_current_gamma = end_time_current_gamma - start_time_current_gamma
                        print('gamma = {}, N = {} took {} sec, rej. rates:'.format(gamma, net_size_list, total_time_current_gamma))
                        print(np.transpose(rej_rate_list_list_list[ll, :, :]))
                    end_time = timer()
                    total_time = end_time - start_time
                    print('N = {} took {} sec.'.format(net_size_list, total_time))
                
                    for ii_stat in range(num_stat_codes):
                        stat_code = stat_code_list[ii_stat]
                        stat_str = stat_str_dict[stat_code]
                        rej_rate_list_list = rej_rate_list_list_list[:, :, ii_stat]
                        #Save result in a text file:
                        file_name_str = '{}gamma-{}-{}-{}_x_min-{}_{}N-{}-{}_stat-{}_surr-{}_rej-{}_{}_nom_size-{}_{}surr_{}tests_safe-{}'.format(len(gamma_list), min(gamma_list), max(gamma_list), gamma, x_min, len(net_size_list), min(net_size_list), max(net_size_list), stat_str, con_typ_str, siz_pow_str, dist_str, nom_size, num_surr, num_tests, 1).replace(".", "-")
                        rel_param_list = [x_min, num_diff_size, num_diff_gamma, stat_code, zero_for_size_one_for_power, dist_code, surr_type_code, nom_size, num_surr, num_tests, start_time, total_time]
                        num_rel_param = len(rel_param_list)
                        length_longest_vec = max(num_diff_gamma, num_diff_size, num_rel_param)
                        data = np.column_stack((list(np.int_(net_size_list)) + [0]*(length_longest_vec - num_diff_size), list(gamma_list) + [0]*(length_longest_vec - num_diff_gamma),  rel_param_list + [0]*(length_longest_vec - num_rel_param)))
                        header = 'Num. nodes, gamma, x_min-num_diff_size-num_diff_gamma-stat_code-zero_for_size_one_for_power-dist_code-surr_type_code-nom_size-num_surr-num_tests-start_time-total_time'
                        np.savetxt('../' + results_folder_name + '/' + file_name_str + '_dir_param_test.dat', data, header=header)
                        np.savetxt('../' + results_folder_name + '/' + file_name_str + '_dir_rej_rates_test.dat', rej_rate_list_list)

L = 16
10 tests, 9 conoplt16tr100000 surrogates, min. deg = 1.
gamma = [2.5], N = [4, 6, 8, 11, 16, 23, 32, 45, 64], stat codes = [0, 1, 2, 3, 4, 20, 21], surr. = conoplt16tr100000, rej. = siz, distrib. = 0.
N = 4 took 31.445520099999996 sec.
[0.  0.1 0.1 0.1 0.  0.1 0. ]
N = 6 took 50.0268279 sec.
[0.  0.2 0.2 0.  0.  0.2 0.1]
N = 8 took 43.05688829999998 sec.
[0.  0.  0.2 0.  0.  0.1 0.3]
N = 11 took 35.82218020000002 sec.
[0.1 0.2 0.  0.1 0.  0.2 0.2]
N = 16 took 41.23016469999999 sec.
[0.  0.  0.  0.3 0.  0.  0.3]
N = 23 took 41.82185599999997 sec.
[0.  0.1 0.3 0.  0.3 0.3 0.1]
N = 32 took 47.963593300000014 sec.
[0.2 0.3 0.2 0.2 0.1 0.1 0.1]
N = 45 took 46.81089750000001 sec.
[0.  0.  0.2 0.  0.1 0.2 0.3]
N = 64 took 48.07110700000004 sec.
[0.2 0.1 0.1 0.1 0.2 0.2 0.1]
gamma = 2.5, N = [4, 6, 8, 11, 16, 23, 32, 45, 64] took 386.2510354 sec, rej. rates:
[[0.  0.  0.  0.1 0.  0.  0.2 0.  0.2]
 [0.1 0.2 0.  0.2 0.  0.1 0.3 0.  0.1]
 [0.1 0.2 0.2 0.  0.  0.3 0.2 0.2 0.1]
 [0.1 0.  0. 

In [7]:
# Generate data for Fig. 5-6, 10-12, S4-6 (using Measures of Analysis of Time Series (MATS) surrogates AAFT and iAAFT)
# 
# Calculate rate of rejection of power-law hypothesis with tests based on a chosen surrogate type.
#
# Make some parameter choices:
L = 16# Length of ordinal patterns
o = 1# Markov order
b = 3# Bin size
num_trans = 10**5# Number of transitions when using constrained Markov or ordinal pattern power-law surrogates
for dist_code in [10]:
    for surr_type_code in [1, 0]:#-2 for shuffle, 0 for constrained, 1 for typical, 17 for constrained ordinal pattern power-law, 20 for contrained Markov order power-law, 101 for AAFT, 102 for iAAFT
        num_tests_list = [10**1]
        nom_size_list = [0.1]; num_surr = 9;
        x_min_list = [1]
        stat_code_list = [0, 1, 2, 3, 4, 20, 21]#KS distance, variance, mean, maximum, conditional entropy, coefficient of variation, order 2 conditional entropy

        # NMax = np.Inf
        NMax = 64
        
        zero_for_size_one_for_power = 1
        
        if dist_code == 0:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);zero_for_size_one_for_power = 0;#Discrete power-law
        elif dist_code == 10:
            gamma_list = np.arange(1.5, 1.5 + 10**-6, 0.5);#Discretised conditioned log-normal distribution with mu = 0
        elif dist_code == 17:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#Power law with exponential cutoff defined by lam = Lambda = lambda = 0.01
        elif dist_code == 25:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); x_min_list = [1];#Discrete power-law which is a Markov chain of order one with a 90% chance that proposed value is in the same bin as the current value
        elif dist_code == 26:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); x_min_list = [1];#Discrete power-law which is a Markov chain of order two with a 90% chance that proposed value is in the same bin as one of the two most recent values
        elif dist_code == 35:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); x_min_list = [1];# = 59 555, Extract of sequential earthquake energies in Southern California
        elif dist_code == 57:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); NMax = 27; x_min_list = [2317]; #N = 27, NG = 27, Extract of tail of rescaled numbers of fatalities in epidemics
        elif dist_code == 58:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); NMax = 57; x_min_list = [235]; #N = 57, NG = 57, Extract of tail of thousands of people affected by blackouts
        elif dist_code == 59:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); NMax = 1711; x_min_list = [1]; #N = 1 711, NG = 1 711, Extract of renormalised tail of peak gamma-ray intensity of solar flares between 1980 and 1989
        elif dist_code == 60:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25); NMax = 318; x_min_list = [2]; #N = 318, NG = 318, Extract of tail of deaths in thousands killed in interstate battles 1816-2014
        elif dist_code == 93:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#Discrete power-law observation of asymmetric tent map with a = 0.95
        elif dist_code == 94:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#Discrete power-law observation of asymmetric tent map with a = 0.98
        elif dist_code == 95:
            gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#Discrete power-law observation of asymmetric tent map with a = 0.99

        if (NMax > 1024):
            NMax = 1024
        numN = int(np.round(np.log2(NMax))) - 1;
        numN = 9

        indNList = 2 + np.arange(0, numN, 1)*(np.log2(NMax) - 2)/(numN - 1)
        net_size_list = [int(np.round(2**n)) for n in indNList]

        num_stat_codes = len(stat_code_list)

        for num_tests in num_tests_list:
            for nom_size in nom_size_list:
                for x_min in x_min_list:
                    # Make more parameter choices:
                    start_time = timer()

                    if (zero_for_size_one_for_power == 0):
                        dist_code = 0;
                        gamma_list = [gamma for gamma in gamma_list if gamma > 1]# The power-law distribution is not defined for scale exponent less than or equal to 1
                        siz_pow_str = 'siz'
                    elif (zero_for_size_one_for_power == 1):
                        siz_pow_str = 'pow'

                    if (surr_type_code == -1):
                        gamma_list = [gamma for gamma in gamma_list if gamma > 1]# The power-law distribution is not defined for scale exponent less than or equal to 1

                    if (surr_type_code == -2):#Shuffle
                        con_typ_str = 'shuff'
                    elif (surr_type_code == 0):#Constrained power-law surrogate
                        con_typ_str = 'const'
                    elif (surr_type_code == 1):#Typical power-law surrogate
                        con_typ_str = 'typic'
                    elif (surr_type_code == 17):#Constrained power-law and also constrained in terms of ordinal patterns of length L (this version allows consideration of elements in the same ordinal pattern)
                        print('L = ' + str(L))
                        con_typ_str = 'conoplt' + str(L) + 'tr' + str(num_trans)
                    elif (surr_type_code == 20):#Constrained power-law and also constrained in terms of relaxed ordinal patterns of length L (this version allows consideration of elements in the same ordinal pattern)
                        print('o = ' + str(o) + ', b = ' + str(b))
                        con_typ_str = 'mcconb' + str(b) + 'o' + str(o) + 'tr' + str(num_trans)
                    elif (surr_type_code == 101):#AAFT
                        con_typ_str = 'aaft'
                    elif (surr_type_code == 102):#AAFT
                        con_typ_str = 'iaaft'

                    gamma_list = list(gamma_list); num_diff_gamma = len(gamma_list)
                    num_diff_size = len(net_size_list);

                    print('{} tests, {} {} surrogates, min. deg = {}.'.format(num_tests, num_surr, con_typ_str, x_min))    
                    print('gamma = {}, N = {}, stat codes = {}, surr. = {}, rej. = {}, distrib. = {}.'.format(gamma_list, net_size_list, stat_code_list, con_typ_str, siz_pow_str, dist_code))

                    #Find rejection rates:
                    rej_rate_list_list_list = np.zeros((num_diff_gamma, num_diff_size, num_stat_codes))
                    for ll in range(num_diff_gamma):
                        gamma = gamma_list[ll]
                        start_time_current_gamma = timer()
                        for kk in range(num_diff_size):
                            N = net_size_list[kk]
                            start_current_num_nodes_time = timer()
                            num_rej_list = np.zeros(num_stat_codes)
                            for jj in range(num_tests):
                                (obs_seq, dist_str) = gen_orig(dist_code, gamma, x_min, N, zero_for_size_one_for_power)
                                gamma_m_l = estim_scale_exp(obs_seq, x_min)
                                obs_stat_list = np.zeros(num_stat_codes)
                                for ii_stat in range(num_stat_codes):
                                    obs_stat_list[ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](obs_seq, x_min, gamma_m_l)
                                surr_stat_list_list = np.zeros((num_surr, num_stat_codes))
                                surr_stat_list = np.zeros(num_surr)
                                if (surr_type_code == -2):
                                    for ii in range(num_surr):
                                        surr_val_seq = list(np.random.permutation(obs_seq))
                                        gamma_m_l_surr = gamma_m_l
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == -1):
                                    for ii in range(num_surr):
                                        surr_val_seq = gen_typ_p_l_surrogate(gamma, x_min, N)
                                        gamma_m_l_surr = estim_scale_exp(surr_val_seq, x_min)
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == 0):
                                    gamma_m_l_surr = gamma_m_l
                                    obs_factor_dict = build_factor_count_dict(obs_seq)
                                    if (x_min == 1):
                                        for ii in range(num_surr):
                                            valid_surr_flag = False
                                            while (not valid_surr_flag):
                                                surr_val_seq = gen_cons_power_law_surr_1(obs_seq, obs_factor_dict)
                                                for ii_stat in range(num_stat_codes):
                                                    surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                                surr_factor_dict = build_factor_count_dict(surr_val_seq)
                                                if not operator.eq(obs_factor_dict, surr_factor_dict):
                                                    set1 = set(obs_factor_dict.items())
                                                    set2 = set(surr_factor_dict.items())
                                                    print('There is a problem: prime factor counts of surrogate do not match those of original. Difference:')
                                                    print(set1 ^ set2)
                                                    print('We will generate a replacement surrogate.')
                                                    valid_surr_flag = False
                                                else:
                                                    valid_surr_flag = True
                                    else:
                                        (move_factor_dict, factor_list, min_admiss_factor_ind_list, val_fix_list) = make_move_factor_dict(obs_factor_dict, obs_seq, x_min)
                                        for ii in range(num_surr):
                                            valid_surr_flag = False
                                            while (not valid_surr_flag):
                                                surr_val_seq = gen_cons_power_law_surr_2(move_factor_dict, factor_list, min_admiss_factor_ind_list, val_fix_list)
                                                for ii_stat in range(num_stat_codes):
                                                    surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                                surr_factor_dict = build_factor_count_dict(surr_val_seq)
                                                if not operator.eq(obs_factor_dict, surr_factor_dict):
                                                    set1 = set(obs_factor_dict.items())
                                                    set2 = set(surr_factor_dict.items())
                                                    print('There is a problem: prime factor counts of surrogate do not match those of original. Difference:')
                                                    print(set1 ^ set2)
                                                    print('We will generate a replacement surrogate.')
                                                    valid_surr_flag = False
                                                else:
                                                    valid_surr_flag = True
                                elif (surr_type_code == 1):
                                    for ii in range(num_surr):
                                        surr_val_seq = gen_typ_p_l_surrogate(gamma_m_l, x_min, N)
                                        gamma_m_l_surr = estim_scale_exp(surr_val_seq, x_min)
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == 17):
                                    gamma_m_l_surr = gamma_m_l
                                    surr_val_seq = obs_seq
                                    for ii in range(num_surr):
                                        surr_val_seq = gen_p_l_markov_o_p_metr_hast_L(surr_val_seq, num_trans, x_min, L)
                                        gamma_m_l_surr = gamma_m_l
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == 20):
                                    bin_edges, bin_seq, low_lim_list, upp_lim_list = log_bin(obs_seq, b, x_min)
                                    surr_val_seq_list = gen_mc_pl_surrogate(obs_seq, bin_edges, num_trans, o, num_surr)
                                    for ii in range(num_surr):
                                        surr_val_seq = surr_val_seq_list[ii]
                                        gamma_m_l_surr = gamma_m_l
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == 101):
                                    surr_val_seq_list = aaft(obs_seq, num_surr)
                                    for ii in range(num_surr):
                                        surr_val_seq = surr_val_seq_list[ii]
                                        gamma_m_l_surr = gamma_m_l#The AAFT surrogate will not change the max. likelihood gamma
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                elif (surr_type_code == 102):
                                    surr_val_seq_list = iaaft(obs_seq, num_surr)
                                    for ii in range(num_surr):
                                        surr_val_seq = surr_val_seq_list[ii]
                                        gamma_m_l_surr = gamma_m_l#The IAAFT surrogate will not change the max. likelihood gamma
                                        for ii_stat in range(num_stat_codes):
                                            surr_stat_list_list[ii, ii_stat] = stat_fun_dict[stat_code_list[ii_stat]](surr_val_seq, x_min, gamma_m_l_surr)
                                for ii_stat in range(num_stat_codes):
                                    surr_stat_list = surr_stat_list_list[:, ii_stat]
                                    stat_factor = stat_factor_dict[stat_code_list[ii_stat]]
                                    obs_stat = np.multiply(stat_factor, obs_stat_list[ii_stat])
                                    abs_obs_stat = abs(obs_stat)
                                    obs_stat = obs_stat + 10**-6*abs_obs_stat*(np.random.uniform(size=1) - 0.5)#Add a small random perturbation to avoid ties
                                    surr_stat_list = np.multiply(stat_factor, surr_stat_list) + 10**-6*abs_obs_stat*(np.random.uniform(size=num_surr) - 0.5)#Add small random perturbations to avoid ties
                                    rankL = sum(surr_stat_list > obs_stat);
                                    rankU = sum(surr_stat_list >= obs_stat)
                                    rank = 1 + random.randint(rankL, rankU)#Randomly assign ranks among tied values
                                    p_val = (rank - 0.5)/(num_surr + 1)
                                    if (p_val <= nom_size):
                                        num_rej_list[ii_stat] += 1
                            for ii_stat in range(num_stat_codes):
                                rej_rate_list_list_list[ll, kk, ii_stat] = num_rej_list[ii_stat]/num_tests
                            end_current_num_nodes_time = timer()
                            if (ll == 0):
                                print('N = {} took {} sec.'.format(N, end_current_num_nodes_time - start_current_num_nodes_time))
                                print(np.transpose(rej_rate_list_list_list[ll, kk, :]))
                        end_time_current_gamma = timer()
                        total_time_current_gamma = end_time_current_gamma - start_time_current_gamma
                        print('gamma = {}, N = {} took {} sec, rej. rates:'.format(gamma, net_size_list, total_time_current_gamma))
                        print(np.transpose(rej_rate_list_list_list[ll, :, :]))
                    end_time = timer()
                    total_time = end_time - start_time
                    print('N = {} took {} sec.'.format(net_size_list, total_time))

                    for ii_stat in range(num_stat_codes):
                        stat_code = stat_code_list[ii_stat]
                        stat_str = stat_str_dict[stat_code]
                        rej_rate_list_list = rej_rate_list_list_list[:, :, ii_stat]
                        #Save result in a text file:
                        file_name_str = '{}gamma-{}-{}-{}_x_min-{}_{}N-{}-{}_stat-{}_surr-{}_rej-{}_{}_nom_size-{}_{}surr_{}tests_safe-{}'.format(len(gamma_list), min(gamma_list), max(gamma_list), gamma, x_min, len(net_size_list), min(net_size_list), max(net_size_list), stat_str, con_typ_str, siz_pow_str, dist_str, nom_size, num_surr, num_tests, 1).replace(".", "-")
                        rel_param_list = [x_min, num_diff_size, num_diff_gamma, stat_code, zero_for_size_one_for_power, dist_code, surr_type_code, nom_size, num_surr, num_tests, start_time, total_time]
                        num_rel_param = len(rel_param_list)
                        length_longest_vec = max(num_diff_gamma, num_diff_size, num_rel_param)
                        data = np.column_stack((list(np.int_(net_size_list)) + [0]*(length_longest_vec - num_diff_size), list(gamma_list) + [0]*(length_longest_vec - num_diff_gamma),  rel_param_list + [0]*(length_longest_vec - num_rel_param)))
                        header = 'Num. nodes, gamma, x_min-num_diff_size-num_diff_gamma-stat_code-zero_for_size_one_for_power-dist_code-surr_type_code-nom_size-num_surr-num_tests-start_time-total_time'
                        np.savetxt('../' + results_folder_name + '/' + file_name_str + '_dir_param_test.dat', data, header=header)
                        np.savetxt('../' + results_folder_name + '/' + file_name_str + '_dir_rej_rates_test.dat', rej_rate_list_list)

10 tests, 9 typic surrogates, min. deg = 1.
gamma = [1.5], N = [4, 6, 8, 11, 16, 23, 32, 45, 64], stat codes = [0, 1, 2, 3, 4, 20, 21], surr. = typic, rej. = pow, distrib. = 10.
N = 4 took 1.5482570999993186 sec.
[0.1 0.  0.  0.  0.1 0.  0. ]
N = 6 took 1.544727899999998 sec.
[0.3 0.  0.  0.  0.  0.1 0.2]
N = 8 took 1.5524067999995168 sec.
[0.1 0.  0.  0.  0.  0.1 0. ]
N = 11 took 1.6050739000002068 sec.
[0.4 0.1 0.1 0.1 0.  0.1 0.1]
N = 16 took 1.5675158000003648 sec.
[0.4 0.1 0.1 0.1 0.  0.1 0.1]
N = 23 took 1.6030315999996674 sec.
[0.4 0.2 0.1 0.3 0.2 0.2 0.2]
N = 32 took 1.5944537000004857 sec.
[0.3 0.4 0.  0.4 0.  0.5 0.1]
N = 45 took 1.6110066000001098 sec.
[0.4 0.4 0.3 0.4 0.  0.6 0. ]
N = 64 took 1.598989399999482 sec.
[0.9 0.6 0.5 0.6 0.  0.7 0.1]
gamma = 1.5, N = [4, 6, 8, 11, 16, 23, 32, 45, 64] took 14.227265200000147 sec, rej. rates:
[[0.1 0.3 0.1 0.4 0.4 0.4 0.3 0.4 0.9]
 [0.  0.  0.  0.1 0.1 0.2 0.4 0.4 0.6]
 [0.  0.  0.  0.1 0.1 0.1 0.  0.3 0.5]
 [0.  0.  0.  0.1 0.1 0.

In [8]:
# Generate data for Fig. 7,S3
# 
# Calculate mean and standard deviation of statistics.
#
# Perform many tests, each with many surrogates of each type.
#

num_tests = 10**1

for gg in [19]:
    x_min_list = [1]
    N_orig = 64
    gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);
    
    if (gg == 0):
        func_flag = 0; gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#0: Power-law
    elif (gg == 1):
        func_flag = 2; gamma_list = np.arange(1.5, 1.5 + 10**-6, 0.25);#Discretised truncated log-normal mu = 0
    elif (gg == 2):
        func_flag = 7; gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#Power law with exponential cut-off 0.01
    elif (gg == 3.5):
        func_flag = 21.5; gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#Discrete power-law truncated at xMax = 64 (When gamma = 2.5 and xMin = 1, the 1/1024 quantile is at k = 64.2388)
        x_min_list = [1]
    elif (gg == 13):
        func_flag = 51;
        file_name_str = 'terrorism.txt'; dist_str = 'terrorism'; x_min_list = [12]; N_orig = 547; #x_min = 12;#+/-4, N = 9,101, NG = 547
    elif (gg == 14):
        func_flag = 53;
        file_name_str = 'rescaled-diseases.txt'; dist_str = 'diseases-rescaled'; x_min_list = [2317]; N_orig = 27; #x_min = 2317, N = 72, NG = 27
    elif (gg == 15):
        func_flag = 55;
        file_name_str = 'blackouts.txt'; dist_str = 'blackouts'; x_min_list = [230];  N_orig = 57; #x_min = 230;#+/-90, N = 211, NG = 57    
    elif (gg == 16):#Upsample or downsample, as required, from data in a text file
        func_flag = 59;
        file_name_str = 'normed-flares.txt'; dist_str = 'flares-normed'; x_min_list = [1]; N_orig = 1711
    elif (gg == 18):#Upsample or downsample, as required, from int_energy_list, a list of energies released by earthquakes
        func_flag = 62;
        dist_str = 'earthquakes'; x_min_list = [1]; N_orig = 59555 #x_min_list = [1, 2]; N_orig_list = [411, 318]
    elif (gg == 19):#Upsample or downsample, as required, from data in a text file
        func_flag = 52
        file_name_str = 'words.txt'; dist_str = 'words'; x_min_list = [7]; N_orig = 958 #x_min = 7;#+/-2, N = 18,855, NG = 958
    
    stat_code_list = [1, 2, 3, 4, 20, 21, 33, 35]#[variance, mean, maximum, conditional entropy, coefficient of variation, variance mean ratio, ratio of largest value to second largest value]
    num_surr = 10**1
    num_diff_size = 9;
    NMax = min(N_orig, 1024); net_size_list = 2.**np.linspace(log(4, 2), log(NMax, 2), num=num_diff_size, endpoint=True)
    net_size_list = [int(np.round(n)) for n in net_size_list]
    num_diff_size = len(net_size_list);

    num_stat_codes = len(stat_code_list)

    for x_min in x_min_list:
        start_time = timer()
        gamma_list = list(gamma_list); num_diff_gamma = len(gamma_list)

        print('{} tests, {} surrogates, min. deg = {}.'.format(num_tests, num_surr, x_min))   
        print('gamma = {}, N = {}, distrib. = {}.'.format(gamma_list, net_size_list, func_flag))
        
        mean_val_list_list_list_list = np.zeros((num_diff_gamma, num_diff_size, num_tests, num_surr_types_eff, num_stat_codes))
        med_val_list_list_list_list = np.zeros((num_diff_gamma, num_diff_size, num_tests, num_surr_types_eff, num_stat_codes))
        std_val_list_list_list_list = np.zeros((num_diff_gamma, num_diff_size, num_tests, num_surr_types_eff, num_stat_codes))
        for ll in range(num_diff_gamma):
            gamma = gamma_list[ll]
            start_time_current_gamma = timer()
            for kk in range(num_diff_size):
                N = net_size_list[kk]
                start_current_num_nodes_time = timer()
                val_list_list_list_list = np.zeros((num_tests, num_surr_types_eff, num_surr, num_stat_codes))
                for jj in range(num_tests):
                    val_list_list_list = calc_val_eff(gamma, stat_fun_dict_a, stat_code_list, x_min, N, func_flag, num_surr)
                    val_list_list_list_list[jj, :, :, :] = val_list_list_list#val_list_list_list_list is [num_tests, num_surr_types_eff, num_surr, num_stats]
                for ii_surr_type in range(num_surr_types_eff):
                    for ii_stat in range(num_stat_codes):
                        mean_val_list_list_list_list[ll, kk, :, ii_surr_type, ii_stat] = np.mean(val_list_list_list_list[:, ii_surr_type, :, ii_stat], axis = 1)#Calculate statistic over different surrogates for each fixed observation
                        med_val_list_list_list_list[ll, kk, :, ii_surr_type, ii_stat] = np.median(val_list_list_list_list[:, ii_surr_type, :, ii_stat], axis = 1)#Calculate statistic over different surrogates for each fixed observation
                        std_val_list_list_list_list[ll, kk, :, ii_surr_type, ii_stat] = np.std(val_list_list_list_list[:, ii_surr_type, :, ii_stat], axis = 1, ddof=1)#Calculate statistic over different surrogates for each fixed observation, sample standard deviation
                end_current_num_nodes_time = timer()
                if (ll == 0):
                    print('Statistic values (N = {}):'.format(net_size_list))
                    print('N = {} took {} sec.'.format(N, end_current_num_nodes_time - start_current_num_nodes_time))
            end_time_current_gamma = timer()
            total_time_current_gamma = end_time_current_gamma - start_time_current_gamma
            print('gamma = {}, N = {} took {} sec, rej. rates:'.format(gamma, net_size_list, total_time_current_gamma))
        end_time = timer()
        total_time = end_time - start_time
        print('N = {} took {} sec.'.format(net_size_list, total_time))

        for ii_surr_type in range(num_surr_types_eff):
            for ii_stat in range(num_stat_codes):
                stat_code = stat_code_list[ii_stat]
                stat_str = stat_str_dict[stat_code]
                mean_val_list_list_list = mean_val_list_list_list_list[:, :, :, ii_surr_type, ii_stat]
                med_val_list_list_list = med_val_list_list_list_list[:, :, :, ii_surr_type, ii_stat]
                std_val_list_list_list = std_val_list_list_list_list[:, :, :, ii_surr_type, ii_stat]
                #Save result in a text file:
                file_name_str = 'stat-vals-{}-tests-{}gamma-{}-{}_x_min-{}_{}N-{}-{}_stat-{}_surr-type-{}_true-func-{}_{}surr_eff'\
                .format(num_tests, len(gamma_list), min(gamma_list), max(gamma_list), x_min, len(net_size_list), min(net_size_list), max(net_size_list), stat_str, ii_surr_type, func_flag, num_surr).replace(".", "-")
                rel_param_list = [x_min, num_surr_types_eff, num_diff_gamma, num_diff_size, stat_code, ii_surr_type, func_flag, num_surr, start_time, total_time]
                num_rel_param = len(rel_param_list)
                length_longest_vec = max(num_diff_gamma, num_surr_types_eff, num_rel_param)
                data = np.column_stack((list(np.int_(net_size_list)) + [0]*(length_longest_vec - num_diff_size), list(gamma_list) + [0]*(length_longest_vec - num_diff_gamma),  rel_param_list + [0]*(length_longest_vec - num_rel_param)))
                header = 'Num. nodes, gamma, x_min-num_surr_types-num_diff_gamma-num_diff_size-stat_code-num_surr_type-func_flag-num_surr-start_time-total_time'
                np.savetxt('../' + results_folder_name + '/' + file_name_str + '_param_test.dat', data, header=header)
                np.save('../' + results_folder_name + '/' + 'mean_' + file_name_str + '_rej_rates_test', mean_val_list_list_list)
                np.save('../' + results_folder_name + '/' + 'med_' + file_name_str + '_rej_rates_test', med_val_list_list_list)
                np.save('../' + results_folder_name + '/' + 'std_' + file_name_str + '_rej_rates_test', std_val_list_list_list)

10 tests, 10 surrogates, min. deg = 7.
gamma = [2.5], N = [4, 8, 16, 31, 62, 123, 244, 483, 958], distrib. = 52.
Statistic values (N = [4, 8, 16, 31, 62, 123, 244, 483, 958]):
N = 4 took 33.059672800000044 sec.
Statistic values (N = [4, 8, 16, 31, 62, 123, 244, 483, 958]):
N = 8 took 33.034288599999854 sec.
Statistic values (N = [4, 8, 16, 31, 62, 123, 244, 483, 958]):
N = 16 took 33.017705000000205 sec.
Statistic values (N = [4, 8, 16, 31, 62, 123, 244, 483, 958]):
N = 31 took 33.28655560000061 sec.
Statistic values (N = [4, 8, 16, 31, 62, 123, 244, 483, 958]):
N = 62 took 33.20559570000023 sec.
Statistic values (N = [4, 8, 16, 31, 62, 123, 244, 483, 958]):
N = 123 took 33.62752019999971 sec.
Statistic values (N = [4, 8, 16, 31, 62, 123, 244, 483, 958]):
N = 244 took 34.778083700000025 sec.
Statistic values (N = [4, 8, 16, 31, 62, 123, 244, 483, 958]):
N = 483 took 36.07043599999997 sec.
Statistic values (N = [4, 8, 16, 31, 62, 123, 244, 483, 958]):
N = 958 took 39.69226500000059 sec.

In [9]:
# Generate data for Fig. 8,9
# 
# Calculate mean and standard deviation of statistics.
#
# Perform many tests, each using one surrogate of each type.
#
for gg in [3.5]:
    x_min_list = [1, 12]; gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);
    N_orig_list = [64]*len(x_min_list)
    dist_str = ''
    
    if (gg == 0):
        func_flag = 0; gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#0: Power-law
    elif (gg == 1):
        func_flag = 2; gamma_list = np.arange(1.5, 1.5 + 10**-6, 0.25);#Discretised truncated log-normal mu = 0
    elif (gg == 2):
        func_flag = 7; gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#Power law with exponential cut-off 0.01
    elif (gg == 3.5):
        func_flag = 21.5; gamma_list = np.arange(2.5, 2.5 + 10**-6, 0.25);#Discrete power-law truncated at xMax = 64 (When gamma = 2.5 and xMin = 1, the 1/1024 quantile is at k = 64.2388)
        x_min_list = [1]
    elif (gg == 13):
        func_flag = 51;
        file_name_str = 'terrorism.txt'; dist_str = 'terrorism'; x_min_list = [1, 12]; N_orig_list = [9101, 547]; #x_min = 12;#+/-4, N = 9,101, NG = 547
    elif (gg == 14):
        func_flag = 53;
        file_name_str = 'rescaled-diseases.txt'; dist_str = 'diseases-rescaled'; x_min_list = [1, 2317]; N_orig_list = [72, 27]; #x_min = 2317, N = 72, NG = 27
    elif (gg == 15):
        func_flag = 55;
        file_name_str = 'blackouts.txt'; dist_str = 'blackouts'; x_min_list = [1, 230];  N_orig_list = [211, 57]; #x_min = 230;#+/-90, N = 211, NG = 57    
    elif (gg == 16):#Upsample or downsample, as required, from data in a text file
        func_flag = 59;
        file_name_str = 'normed-flares.txt'; dist_str = 'flares-normed'; x_min_list = [1]; N_orig_list = [1711]
    elif (gg == 18):#Upsample or downsample, as required, from int_energy_list, a list of energies released by earthquakes
        func_flag = 62;
        dist_str = 'earthquakes'; x_min_list = [1]; N_orig_list = [59555] #x_min_list = [1, 2]; N_orig_list = [411, 318]
    elif (gg == 19):#Upsample or downsample, as required, from data in a text file
        func_flag = 52
        file_name_str = 'words.txt'; dist_str = 'words'; x_min_list = [7]; N_orig_list = [958] #x_min = 7;#+/-2, N = 18,855, NG = 958
        
    num_tests = 10**1
    
    stat_code_list = [1, 2, 3, 4]
    num_surr = 1
    num_diff_size = 9;

    num_stat_codes = len(stat_code_list)

    for ii_deg in range(len(x_min_list)):
        x_min = x_min_list[ii_deg]
        # Make more parameter choices:
        start_time = timer()
        gamma_list = list(gamma_list); num_diff_gamma = len(gamma_list)
        NMax = min(N_orig_list[ii_deg], 1024); net_size_list = 2.**np.linspace(log(4, 2), log(NMax, 2), num=num_diff_size, endpoint=True)
        net_size_list = [int(np.round(n)) for n in net_size_list]
        num_diff_size = len(net_size_list);
        print('{} tests, min. deg = {}.'.format(num_tests, x_min))
        print('gamma = {}, N = {}, distrib. = {}.'.format(gamma_list, net_size_list, func_flag))

        mean_val_list_list_list_list = np.zeros((num_diff_gamma, num_diff_size, num_surr_types_eff, num_stat_codes))
        med_val_list_list_list_list = np.zeros((num_diff_gamma, num_diff_size, num_surr_types_eff, num_stat_codes))
        std_val_list_list_list_list = np.zeros((num_diff_gamma, num_diff_size, num_surr_types_eff, num_stat_codes))
        for ll in range(num_diff_gamma):
            gamma = gamma_list[ll]
            start_time_current_gamma = timer()
            for kk in range(num_diff_size):
                N = net_size_list[kk]
                start_current_num_nodes_time = timer()
                val_list_list_list_list = np.zeros((num_tests, num_surr_types_eff, num_surr, num_stat_codes))
                for jj in range(num_tests):
                    val_list_list_list = calc_val_eff(gamma, stat_fun_dict_a, stat_code_list, x_min, N, func_flag, num_surr)
                    #val_list_list_list is [num_surr_types_eff, num_surr, num_stats]
                    val_list_list_list_list[jj, :, :, :] = val_list_list_list#val_list_list_list_list is [num_tests, num_surr_types_eff, num_surr, num_stats]
                for ii_surr_type in range(num_surr_types_eff):
                    for ii_stat in range(num_stat_codes):
                        mean_val_list_list_list_list[ll, kk, ii_surr_type, ii_stat] = np.mean(val_list_list_list_list[:, ii_surr_type, 0, ii_stat])
                        med_val_list_list_list_list[ll, kk, ii_surr_type, ii_stat] = np.median(val_list_list_list_list[:, ii_surr_type, 0, ii_stat])
                        std_val_list_list_list_list[ll, kk, ii_surr_type, ii_stat] = np.std(val_list_list_list_list[:, ii_surr_type, 0, ii_stat], ddof=1)#Sample standard deviation
                end_current_num_nodes_time = timer()
                if (ll == 0):
                    print('N = {} took {} sec.'.format(N, end_current_num_nodes_time - start_current_num_nodes_time))
            end_time_current_gamma = timer()
            total_time_current_gamma = end_time_current_gamma - start_time_current_gamma
            print('gamma = {}, N = {} took {} sec, rej. rates:'.format(gamma, net_size_list, total_time_current_gamma))
        end_time = timer()
        total_time = end_time - start_time
        print('N = {} took {} sec.'.format(net_size_list, total_time))

        for ii_surr_type in range(num_surr_types_eff):
            for ii_stat in range(num_stat_codes):
                stat_code = stat_code_list[ii_stat]
                stat_str = stat_str_dict[stat_code]
                mean_val_list_list_list = mean_val_list_list_list_list[:, :, ii_surr_type, ii_stat]
                med_val_list_list_list = med_val_list_list_list_list[:, :, ii_surr_type, ii_stat]
                std_val_list_list_list = std_val_list_list_list_list[:, :, ii_surr_type, ii_stat]
                #Save result in a text file:
                file_name_str = ('stat-vals-1-surr-{}gamma-{}-{}_x_min-{}_{}N-{}-{}_stat-{}_surr-type-{}_true-func-{}' + dist_str + '_{}tests_eff')\
                .format(len(gamma_list), min(gamma_list), max(gamma_list), x_min, len(net_size_list), min(net_size_list), max(net_size_list), stat_str, ii_surr_type, func_flag, num_tests).replace(".", "-")
                rel_param_list = [x_min, num_surr_types_eff, num_diff_gamma, num_diff_size, stat_code, ii_surr_type, func_flag, num_tests, start_time, total_time]
                num_rel_param = len(rel_param_list)
                length_longest_vec = max(num_diff_gamma, num_surr_types_eff, num_rel_param)
                data = np.column_stack((list(np.int_(net_size_list)) + [0]*(length_longest_vec - num_diff_size), list(gamma_list) + [0]*(length_longest_vec - num_diff_gamma),  rel_param_list + [0]*(length_longest_vec - num_rel_param)))
                header = 'Num. nodes, gamma, x_min-num_surr_types_eff-num_diff_gamma-num_diff_size-stat_code-num_surr_type-func_flag-num_surr-num_tests-start_time-total_time'
                np.savetxt('../' + results_folder_name + '/' + file_name_str + '_param_test.dat', data, header=header)
                np.savetxt('../' + results_folder_name + '/' + 'mean_' + file_name_str + '_rej_rates_test.dat', mean_val_list_list_list)
                np.savetxt('../' + results_folder_name + '/' + 'med_' + file_name_str + '_rej_rates_test.dat', med_val_list_list_list)
                np.savetxt('../' + results_folder_name + '/' + 'std_' + file_name_str + '_rej_rates_test.dat', std_val_list_list_list)

10 tests, min. deg = 1.
gamma = [2.5], N = [4, 6, 8, 11, 16, 23, 32, 45, 64], distrib. = 21.5.
N = 4 took 0.19333869999991293 sec.
N = 6 took 0.19012900000052468 sec.
N = 8 took 0.21439749999990454 sec.
N = 11 took 0.18950310000036552 sec.
N = 16 took 0.2045348000001468 sec.
N = 23 took 0.19949540000015986 sec.
N = 32 took 0.19453659999999218 sec.
N = 45 took 0.20909060000030877 sec.
N = 64 took 0.21015220000026602 sec.
gamma = 2.5, N = [4, 6, 8, 11, 16, 23, 32, 45, 64] took 1.8056778999998642 sec, rej. rates:
N = [4, 6, 8, 11, 16, 23, 32, 45, 64] took 1.8058668000003308 sec.


In [10]:
# Generate data for Fig. S1
# 
# Prepare histograms comparing original and surrogate marginal distributions
# Generate a single synthetic power-law data sets
#     From this synthetic data set, generate many surrogates from this empirical data using different methods
# 
# 
# Generate i.i.d. power-law sequence
N = 64
x_min = 1

num_surr = 10**1
scale_exp = 2.5#Known scale exponent (Known scale exponent "know" surrogates only)
o = 1#Markov order (Markov order surrogates "mark" only)
b = 3#Use bins of logarithmic width log(b) (Markov order surrogates "mark" only)
L = 16#Length of ordinal patterns (ordinal pattern surrogates "ordi" only)
num_trans = 10**5#Number of transitions (Markov order "mark" and ordinal pattern surrogates "ordi" only)

seq = [0 for i in range(N)]
seq = gen_power_law_surr_list(seq, surr_method='know', x_min=x_min, num_surr=1, scale_exp=scale_exp)[0]

#The list of surrogates to be considered, and their names
surr_method_list = ['obse', 'know', 'typi', 'cons', 'boot', 'shuf', 'mark1', 'mark2', 'ordi', 'aaft', 'iaaft']
surr_method_name_list = ['Observed', 'True', 'Typical', 'Constrained', 'Bootstrapped', 'Shuffled', 'Markov 1', 'Markov 2', 'Ordinal pattern', 'AAFT', 'IAAFT']

#Plot realisation of each type of surrogate
num_surr_types = len(surr_method_list)
surr_val_seq_list_list = []
for i in range(num_surr_types):
    surr_method = surr_method_list[i]
    if (surr_method[0:4] == 'mark'):
        o = int(surr_method[4])
        surr_method = surr_method[0:4]
    surr_val_seq_list = gen_power_law_surr_list(seq, surr_method=surr_method, x_min=x_min, num_surr=num_surr, scale_exp=scale_exp, o=o, b=b, L=L, num_trans=num_trans)
    surr_val_seq_list_list = surr_val_seq_list_list + [surr_val_seq_list]

save_name = 'surr_' + 'N-' + str(N) + '_gamma-' + str(scale_exp) + '_x-min-' + str(x_min) +'_' + str(num_surr_types) + 'surr-types_' + str(num_surr) + 'realis_' + 'b-' + str(b) + '_L-' + str(L) + '_tran-' + str(num_trans) + '.json'

json.dump([N, scale_exp, x_min, seq, num_surr, b, L, num_trans, surr_method_list, surr_method_name_list, surr_val_seq_list_list, save_name], codecs.open('../' + results_folder_name + '/' + save_name, 'w', encoding='utf-8'))

In [11]:
## Generate results for Fig. S2
##
# Calculate surrogate time series from which we will later calculate the conditional entropy
# Generate many synthetic power-law data sets
#     For each synthetic data set, generate a single surrogate from this empirical data using different methods
# 
# 
# Generate i.i.d. power-law sequence
N = 64
x_min = 1
num_trials = 10**1
num_surr = 10**1
scale_exp = 2.5#Known scale exponent (Known scale exponent "know" surrogates only)
o = 1#Markov order (Markov order surrogates "mark" only)
b = 3#Use bins of logarithmic width log(b) (Markov order surrogates "mark" only)
L = 16#Length of ordinal patterns (ordinal pattern surrogates "ordi" only)
num_trans = 10**5#Number of transitions (Markov order "mark" and ordinal pattern surrogates "ordi" only)

seq_list = []
surr_val_seq_list_list_list = []

for j in range(num_trials):

    seq = [0 for i in range(N)]
    seq = gen_power_law_surr_list(seq, surr_method='know', x_min=x_min, num_surr=1, scale_exp=scale_exp)[0]
    seq_list = seq_list + [seq]

    #The list of surrogates to be considered, and their names
    surr_method_list = ['obse', 'know', 'typi', 'cons', 'boot', 'shuf', 'mark1', 'mark2', 'ordi', 'aaft', 'iaaft']
    surr_method_name_list = ['Observed', 'True', 'Typical', 'Constrained', 'Bootstrapped', 'Shuffled', 'Markov 1', 'Markov 2', 'Ordinal pattern', 'AAFT', 'IAAFT']

    #Plot realisation of each type of surrogate
    num_surr_types = len(surr_method_list)
    surr_val_seq_list_list = []
    for i in range(num_surr_types):
        surr_method = surr_method_list[i]
        if (surr_method[0:4] == 'mark'):
            o = int(surr_method[4])
            surr_method = surr_method[0:4]
        surr_val_seq_list = gen_power_law_surr_list(seq, surr_method=surr_method, x_min=x_min, num_surr=num_surr, scale_exp=scale_exp, o=o, b=b, L=L, num_trans=num_trans)
        surr_val_seq_list_list = surr_val_seq_list_list + [surr_val_seq_list]
    surr_val_seq_list_list_list = surr_val_seq_list_list_list + [surr_val_seq_list_list]
        
save_name = 'surr_' + str(num_trials) + '_trials_' + 'N-' + str(N) + '_gamma-' + str(scale_exp) + '_x-min-' + str(x_min) +'_' + str(num_surr_types) + 'surr-types_' + str(num_surr) + 'realis_' + 'b-' + str(b) + '_L-' + str(L) + '_tran-' + str(num_trans) + '.json'

json.dump([N, scale_exp, x_min, seq_list, num_surr, b, L, num_trans, surr_method_list, surr_method_name_list, surr_val_seq_list_list_list, save_name], codecs.open('../' + results_folder_name + '/' + save_name, 'w', encoding='utf-8'))

In [12]:
## Generate results for Fig. S6
# 
# Determine time required to generate different surrogates, using a fixed number of transitions per surrogate
# 
# 
num_trials = 10**1#Number of trials
num_surr = 10**1#Number of surrogates per trial

# Generate i.i.d. power-law sequence
# NN = [4, 8, 16, 32, 64, 128, 256, 512, 1024]
NN = [4, 8, 16, 32]
x_min = 1
num_trials = 10**1
num_surr = 1

scale_exp = 2.5#Known scale exponent (Known scale exponent "know" surrogates only)
o = 1#Markov order (Markov order surrogates "mark" only)
b = 3#Use bins of logarithmic width log(b) (Markov order surrogates "mark" only)
L = 16#Length of ordinal patterns (ordinal pattern surrogates "ordi" only)
num_trans = 10**5#Number of transitions (Markov order "mark" and ordinal pattern surrogates "ordi" only)

#The list of surrogates to be considered, and their names
surr_method_list = ['obse', 'know', 'typi', 'cons', 'boot', 'shuf', 'mark1', 'mark2', 'ordi', 'aaft', 'iaaft']
surr_method_name_list = ['Observed', 'True', 'Typical', 'Constrained', 'Bootstrapped', 'Shuffled', 'Markov 1', 'Markov 2', 'Ordinal pattern', 'AAFT', 'IAAFT']
num_surr_types = len(surr_method_list)

#Generate and plot realisation of each type of surrogate
time_list_list = []
for N in NN:
    time_list = [0 for i in range(num_surr_types)]
    for j in range(num_trials):
        seq = [0 for i in range(N)]
        seq = gen_power_law_surr_list(seq, surr_method='know', x_min=x_min, num_surr=1, scale_exp=scale_exp)[0]
        for i in range(num_surr_types):
            surr_method = surr_method_list[i]
            if (surr_method[0:4] == 'mark'):
                o = int(surr_method[4])
                surr_method = surr_method[0:4]
            init_time = timer()
            surr_val_seq_list = gen_power_law_surr_list(seq, surr_method=surr_method, x_min=x_min, num_surr=num_surr, scale_exp=scale_exp, o=o, b=b, L=L, num_trans=num_trans)
            final_time = timer()
            time = final_time - init_time
            time_list[i] = time_list[i] + time
    time_list_list = time_list_list + [time_list]

save_name = 'time_' + str(len(NN)) + 'N-' + str(min(NN)) + '-' + str(max(NN)) + '_gamma-' + str(scale_exp) + '_x-min-' + str(x_min) +'_' + str(num_surr_types) + 'surr-types_' + str(num_trials) + 'trials_' + str(num_surr) + 'realis_' + 'b-' + str(b) + '_L-' + str(L) + '_tran-' + str(num_trans) + '.json'

json.dump([NN, scale_exp, x_min, seq, num_surr, num_trials, b, L, num_trans, surr_method_list, surr_method_name_list, time_list_list, save_name], codecs.open('../' + results_folder_name + '/' + save_name, 'w', encoding='utf-8'))

In [13]:
## Generate results for Fig. S6
# 
# Determine time required to generate different surrogates, using N^2 transitions per surrogate
# 
# 
num_trials = 10**1#Number of trials
num_surr = 10**1#Number of surrogates per trial

# Generate i.i.d. power-law sequence
# NN = [4, 8, 16, 32, 64, 128, 256, 512, 1024]
NN = [4, 8, 16, 32]
x_min = 1
num_trials = 10**1
num_surr = 1

scale_exp = 2.5#Known scale exponent (Known scale exponent "know" surrogates only)
o = 1#Markov order (Markov order surrogates "mark" only)
b = 3#Use bins of logarithmic width log(b) (Markov order surrogates "mark" only)
L = 16#Length of ordinal patterns (ordinal pattern surrogates "ordi" only)
num_trans = 10**5#Number of transitions (Markov order "mark" and ordinal pattern surrogates "ordi" only)

#The list of surrogates to be considered, and their names
surr_method_list = ['obse', 'know', 'typi', 'cons', 'boot', 'shuf', 'mark1', 'mark2', 'ordi', 'aaft', 'iaaft']
surr_method_name_list = ['Observed', 'True', 'Typical', 'Constrained', 'Bootstrapped', 'Shuffled', 'Markov 1', 'Markov 2', 'Ordinal pattern', 'AAFT', 'IAAFT']
num_surr_types = len(surr_method_list)

#Generate and plot realisation of each type of surrogate
time_list_list = []
for N in NN:
    time_list = [0 for i in range(num_surr_types)]
    for j in range(num_trials):
        seq = [0 for i in range(N)]
        seq = gen_power_law_surr_list(seq, surr_method='know', x_min=x_min, num_surr=1, scale_exp=scale_exp)[0]
        for i in range(num_surr_types):
            surr_method = surr_method_list[i]
            if (surr_method[0:4] == 'mark'):
                o = int(surr_method[4])
                surr_method = surr_method[0:4]
            init_time = timer()
            surr_val_seq_list = gen_power_law_surr_list(seq, surr_method=surr_method, x_min=x_min, num_surr=num_surr, scale_exp=scale_exp, o=o, b=b, L=L, num_trans=N**2)
            final_time = timer()
            time = final_time - init_time
            time_list[i] = time_list[i] + time
    time_list_list = time_list_list + [time_list]

save_name = 'time_' + str(len(NN)) + 'N-' + str(min(NN)) + '-' + str(max(NN)) + '_gamma-' + str(scale_exp) + '_x-min-' + str(x_min) +'_' + str(num_surr_types) + 'surr-types_' + str(num_trials) + 'trials_' + str(num_surr) + 'realis_' + 'b-' + str(b) + '_L-' + str(L) + '_tran-N^2' + '.json'

json.dump([NN, scale_exp, x_min, seq, num_surr, num_trials, b, L, num_trans, surr_method_list, surr_method_name_list, time_list_list, save_name], codecs.open('../' + results_folder_name + '/' + save_name, 'w', encoding='utf-8'))